In [ ]:
# loded the dev_train, dev_test data
# add all features, what i have done before,
# test directly the dev_test data on the rfc_us model
# Under-Sampling	0.7507	0.5165	0.5341	0.5251 
#  1                 .76    0.72      0.48      0.58  

In [1]:
import pandas as pd
import polars as pl
# from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import string
import nltk
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import pipeline
# import pandas as pd
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
# import numpy as np
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report

from sklearn.decomposition import PCA
import joblib

import spacy
from collections import Counter
import numpy as np

from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

from pprint import pprint 


# 1. Load Data Set

In [4]:
# new_df = pd.read_csv('created_new_train_data/new_10.csv') # checkworthy_english_train dataset with new created features
dev_df = pd.read_csv('data/CT24_checkworthy_english/CT24_checkworthy_english/CT24_checkworthy_english_dev.tsv', delimiter = '\t') # validation during training
dev_test_df = pd.read_csv('data/CT24_checkworthy_english/CT24_checkworthy_english/CT24_checkworthy_english_dev-test.tsv', delimiter = '\t') # validation for fine tuning

In [ ]:
# print(dev_df[dev_df['class_label'] == 'Yes'].count()) # 238
# print(dev_df[dev_df['class_label'] == 'No'].count()) # 794

# print(dev_test_df[dev_test_df['class_label'] == 'Yes'].count()) # 108
# print(dev_test_df[dev_test_df['class_label'] == 'No'].count()) # 210

In [5]:
# new label from text to 0/1
dev_df['label'] = dev_df['class_label'].apply(lambda x: 1 if x == 'Yes' else 0)

In [6]:
print(new_df.shape)
print(dev_df.shape)
print(dev_test_df.shape)

(22501, 38)
(1032, 4)
(318, 3)


## Adding new features in the dev dataset

In [7]:
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\looka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\looka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 2. Generate Tokens

In [8]:
def clean_text(text):
    # remove punctuation
    text = ''.join([char for char in text if char not in string.punctuation]) # 
    # tokenize
    tokens = word_tokenize(text)
    # remove stop words
    token = [word for word in tokens if word.lower() not in stop_words]
    return token

In [9]:
# creating new column name Token
dev_df['Tokens'] = dev_df['Text'].apply(clean_text)

# 3. Row text length

In [10]:
def verious_text_length(text):
    text_length = len(text)
    return text_length

In [11]:
dev_df['text_length'] = dev_df['Text'].apply(verious_text_length)

In [12]:
def cleaned_text(text):
    # lowercase the data
    lower_text = text.lower()
    # removing the punctuations
    remove_punct = ''.join([char for char in lower_text if char not in string.punctuation])
    # remove the stop words
    words = word_tokenize(remove_punct)
    remove_stop_words = [word for word in words if word not in stop_words]
    # removing the numbers
    # removing the extra space
    # replace the repetations of punctuations
    # removing emojis
    # removing emoticons
    # removing contractions
    
    return ' '.join(remove_stop_words)

# 4. clean text and cleaned text length

In [13]:
dev_df['cleaned_text'] = dev_df['Text'].apply(cleaned_text)

In [14]:
dev_df['cleaned_text_length'] = dev_df['cleaned_text'].apply(verious_text_length)

In [15]:
dev_df.sample(2)

,Sentence_id,Text,class_label,label,Tokens,text_length,cleaned_text,cleaned_text_length
299,7145,That's not right in America.,No,0,"[Thats, right, America]",28,thats right america,19
310,7346,"I agree, we shouldn't have quotas.",No,0,"[agree, shouldnt, quotas]",34,agree shouldnt quotas,21


In [16]:
# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

C:\Users\looka\OneDrive\Documents\Thesis\master_thesis\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\looka\.cache\huggingface\hub\models--cardiffnlp--twitter-roberta-base-sentiment. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [17]:
def get_sentiment_features(text):
    # Tokenize the input text
    tokens = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Perform inference
    outputs = model(**tokens)
    logits = outputs.logits
    
    # Apply softmax to calculate probabilities
    probs = torch.nn.functional.softmax(logits, dim=-1)
    negative, neutral, positive = probs[0].tolist()
    
    # Define thresholds for "mixed" classification (adjust as needed)
    if max([positive, negative, neutral]) == positive and positive > 0.5:
        sentiment = "Positive"
    elif max([positive, negative, neutral]) == negative and negative > 0.5:
        sentiment = "Negative"
    else:
        sentiment = "Mixed"
    
    return {
        "positive_score": positive,
        "negative_score": negative,
        "neutral_score": neutral,
        "sentiment_class": sentiment
    }

# 5. roberta sentiment generate, then different sentiment put in seperate coulumn

In [18]:
dev_df['roberta_sentiment'] = dev_df['Text'].apply(get_sentiment_features) 

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

In [19]:
dev_df["roberta_sent_pos"] = dev_df["roberta_sentiment"].apply(lambda x: x["positive_score"])
dev_df["roberta_sent_neg"] = dev_df["roberta_sentiment"].apply(lambda x: x["negative_score"])
dev_df["roberta_sent_mixed"] = dev_df["roberta_sentiment"].apply(lambda x: x["neutral_score"])
dev_df["roberta_sentiment_class"] = dev_df["roberta_sentiment"].apply(lambda x: x["sentiment_class"])

In [21]:
def count_tokens(tokens):
    return len(tokens)

# 6. token length

In [22]:
dev_df['token_length'] = dev_df['Tokens'].apply(count_tokens)

In [ ]:
# ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english", grouped_entities=True)

In [24]:
nlp = spacy.load("en_core_web_sm")

In [25]:
def count_entities_to_columns(text):
    

    # # Process each row in the DataFrame
    # for text in df[text_column]:
    #     # Default counts for each row
    names = 0
    orgs = 0
    dates = 0

        # Process text using spaCy
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            names += 1
        elif ent.label_ == "ORG":
            orgs += 1
        elif ent.label_ == "DATE":
            dates += 1

    return {
        "names": names,
        "organizations": orgs,
        "dates": dates
    }

# 7. count name organization and date, then out in the seperate columns

In [27]:
dev_df['counts'] = dev_df['Text'].apply(count_entities_to_columns)

In [29]:
dev_df["names"] = dev_df["counts"].apply(lambda x: x["names"])
dev_df["organizations"] = dev_df["counts"].apply(lambda x: x["organizations"])
dev_df["dates"] = dev_df["counts"].apply(lambda x: x["dates"])
# df_new["sentiment_class"] = df_new["roberta_sentiment"].apply(lambda x: x["sentiment_class"])

In [30]:
def count_punctuation(text):
    # Define a regular expression pattern to match punctuation characters
    punctuation_pattern = r'[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]'
    # Find all punctuation marks in the text
    punctuation_count = len(re.findall(punctuation_pattern, text))
    return punctuation_count

# 8. count punctuations

In [31]:
dev_df['punctuation_count'] = dev_df['Text'].apply(count_punctuation)

In [97]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

In [98]:
def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
    output = model(**inputs)
    embedding = output.last_hidden_state[:, 0, :]
    return embedding.detach().numpy()

# 9. generate embeddings

In [34]:
dev_df['embedding'] = dev_df['Text'].apply(get_embeddings)

In [35]:
# convert to numpy array
def convert_to_numpy_array(embd_str):
    #print(f"string: {embd_str}")
    #print(type(embd_str))
    # embd_str_clean = embd_str.replace("\n", " ").replace(" ", " ")
    #print(type(embd_str_clean))
    #print(f"string cleansing: {embd_str_clean}")
    # embd_str_clean = embd_str.strip('[]')
    #print(type(embd_str_clean))
    #print(f"removing the outer brackets {embd_str_clean}")
    #embd_list = embd_str.split()
    #print(f"Split the string into individual values {embd_list}")
    embed_array = np.array(embd_str, dtype = float)
    #print(f"list to array {embed_array}")
    return embed_array

# 10. renaming the column name , similler to the column name, when the model was trained

In [36]:
dev_df.rename(columns = {'token_length': 'count_tokens'}, inplace = True)

# 11. convert the embedding into numpy array for model input

In [37]:
dev_df['converted_embedding'] = dev_df['embedding'].apply(convert_to_numpy_array)

In [38]:
dev_df = dev_df.astype({'text_length': 'int32', 'roberta_sent_neg': 'float32', 'roberta_sent_mixed': 'float32', 'roberta_sent_pos': 'float32',
               'names': 'int32', 'organizations': 'int32', 'dates': 'int32', 'count_tokens': 'int32', 'cleaned_text_length': 'int32', 'punctuation_count': 'int32'})

In [39]:
print(dev_df['converted_embedding'].iloc[0].shape)

(1, 768)


# 12. flatten the embedding

In [40]:
dev_df['flattened_embedding'] = dev_df['converted_embedding'].apply(lambda x: np.squeeze(x))

In [52]:
dev_df.to_csv('created_new_dev_data/first_dev.csv', index = False)

In [46]:
print(dev_df['embedding'].iloc[0].shape)
print(dev_df['converted_embedding'].iloc[0].shape)
print(dev_df['flattened_embedding'].iloc[0].shape)

(1, 768)
(1, 768)
(768,)


In [49]:
#rf = RandomForestRegressor(random_state = 42)

In [57]:
# print("Parameters are currently in used:\n")
# pprint(rf.get_params())

<!-- # n_estimators - number of tree in the forest
# max_features - max number of features considered for splitting a node 
# max_depth - max number of leveled in each decicion tree
# min_samples_split - min number of adta points placed in a node before the node is split
# min_samples_leaf - min number of data points allowed in a leaf node
# bootstrap - methos for resampling data points (with or without replacement) -->

<!-- n_estimators = number of trees in the foreset
max_features = max number of features considered for splitting a node
max_depth = max number of levels in each decision tree
min_samples_split = min number of data points placed in a node before the node is split
min_samples_leaf = min number of data points allowed in a leaf node
bootstrap = method for sampling data points (with or without replacement) -->

## n_estimators = number of trees in the foreset
## max_features = max number of features considered for splitting a node
## max_depth = max number of levels in each decision tree
## min_samples_split = min number of data points placed in a node before the node is split
## min_samples_leaf = min number of data points allowed in a leaf node
## bootstrap = method for sampling data points (with or without replacement)

In [53]:
# loading the best model so far
# load model rfc us
# trained on original train data (not dev data)
model_path_rfc_us = 'models/random_forest_tfidf_embeddings_10_features_us.pkl'
model_rfc_us = joblib.load(model_path_rfc_us)
# predictions_rfc_us = model_rfc_us.predict(X)

In [55]:
model_rfc_us.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [56]:
best_params = {
     'n_estimators': 800,
     'min_samples_split': 5,
     'min_samples_leaf': 1,
     'max_features': 'sqrt',
     'max_depth': 90,
     'bootstrap': False
}

# first test the dev_test data with the base best model, classification report

In [118]:

dev_test_df.sample(1)

,Sentence_id,Text,class_label,label,Tokens,text_length,cleaned_text,cleaned_text_length,roberta_sentiment,count_tokens,counts,roberta_sent_pos,roberta_sent_neg,roberta_sent_mixed,roberta_sentiment_class,names,organizations,dates,punctuation_count,flattened_embedding
114,38863,We take care of our vets.,No,0,"[take, care, vets]",25,take care vets,14,"{'positive_score': 0.3156950771808624, 'negati...",3,"{'names': 0, 'organizations': 0, 'dates': 0}",0.315695,0.092554,0.591751,Mixed,0,0,0,0,"[0.005399604327976704, 0.6046665906906128, -0...."


In [131]:
dev_test_df.columns

Index(['Sentence_id', 'Text', 'class_label', 'label', 'Tokens', 'text_length',
       'cleaned_text', 'cleaned_text_length', 'roberta_sentiment',
       'count_tokens', 'counts', 'roberta_sent_pos', 'roberta_sent_neg',
       'roberta_sent_mixed', 'roberta_sentiment_class', 'names',
       'organizations', 'dates', 'punctuation_count', 'flattened_embedding'],
      dtype='object')

## Add the same features in the dev_test_df for testing the model (model_rfc_us) on it

In [134]:
dev_test = pd.read_csv('created_new_dev_data/first_dev_test.csv')
dev_train = pd.read_csv('created_new_dev_data/first_dev_train.csv')

In [140]:
print(dev_train['label'].value_counts())
print(dev_test['label'].value_counts())

label
0    794
1    238
Name: count, dtype: int64
label
0    210
1    108
Name: count, dtype: int64


In [75]:
dev_test_df['label'] = dev_test_df['class_label'].apply(lambda x: 1 if x == 'Yes' else 0)

In [77]:
dev_test_df['Tokens'] = dev_test_df['Text'].apply(clean_text)

In [78]:
dev_test_df['text_length'] = dev_test_df['Text'].apply(verious_text_length)

In [79]:
dev_test_df['cleaned_text'] = dev_test_df['Text'].apply(cleaned_text)

In [80]:
dev_test_df['cleaned_text_length'] = dev_test_df['cleaned_text'].apply(verious_text_length)

In [90]:
dev_test_df['roberta_sentiment'] = dev_test_df['Text'].apply(get_sentiment_features) 

In [93]:
dev_test_df["roberta_sent_pos"] = dev_test_df["roberta_sentiment"].apply(lambda x: x["positive_score"])
dev_test_df["roberta_sent_neg"] = dev_test_df["roberta_sentiment"].apply(lambda x: x["negative_score"])
dev_test_df["roberta_sent_mixed"] = dev_test_df["roberta_sentiment"].apply(lambda x: x["neutral_score"])
dev_test_df["roberta_sentiment_class"] = dev_test_df["roberta_sentiment"].apply(lambda x: x["sentiment_class"])

In [91]:
dev_test_df['token_length'] = dev_test_df['Tokens'].apply(count_tokens)

In [92]:
dev_test_df['counts'] = dev_test_df['Text'].apply(count_entities_to_columns)

In [94]:
dev_test_df["names"] = dev_test_df["counts"].apply(lambda x: x["names"])
dev_test_df["organizations"] = dev_test_df["counts"].apply(lambda x: x["organizations"])
dev_test_df["dates"] = dev_test_df["counts"].apply(lambda x: x["dates"])

In [95]:
dev_test_df['punctuation_count'] = dev_test_df['Text'].apply(count_punctuation)

In [99]:
dev_test_df['embedding'] = dev_test_df['Text'].apply(get_embeddings)

In [100]:
dev_test_df.rename(columns = {'token_length': 'count_tokens'}, inplace = True)

In [101]:
dev_test_df['converted_embedding'] = dev_test_df['embedding'].apply(convert_to_numpy_array)

In [102]:
dev_test_df = dev_test_df.astype({'text_length': 'int32', 'roberta_sent_neg': 'float32', 'roberta_sent_mixed': 'float32', 'roberta_sent_pos': 'float32',
               'names': 'int32', 'organizations': 'int32', 'dates': 'int32', 'count_tokens': 'int32', 'cleaned_text_length': 'int32', 'punctuation_count': 'int32'})

In [103]:
dev_test_df['flattened_embedding'] = dev_test_df['converted_embedding'].apply(lambda x: np.squeeze(x))

In [104]:
dev_test_df.to_csv('created_new_dev_data/first_dev_test.csv', index = False)

In [114]:
np.shape(dev_test_df['flattened_embedding'].iloc[0] )# flatten embedding == converted_embedding and shape of both is equal (from checkworthy_2)

(768,)

In [117]:
# because flatten embedidng is looked like converted embedding (from checkworthy_2) and dropped both columns are same
dev_test_df.drop(['converted_embedding', 'embedding'], axis = 1, inplace = True)

In [119]:
dev_test_df[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']]

dev_test_df = dev_test_df.astype({'text_length': 'int32', 'roberta_sent_neg': 'float32', 'roberta_sent_mixed': 'float32', 'roberta_sent_pos': 'float32',
               'names': 'int32', 'organizations': 'int32', 'dates': 'int32', 'count_tokens': 'int32', 'cleaned_text_length': 'int32', 'punctuation_count': 'int32'})

In [121]:
pca = PCA(n_components = 50)
reduced_embeddings = pca.fit_transform(dev_test_df['flattened_embedding'].to_list())

In [124]:
text = dev_test_df['Text']
vectorizer_path = 'models/tfidf_vectorizer_10000.pkl'
tfidf_vect = joblib.load(vectorizer_path) ## loading the trained tfidf vectorizer
tfidf_features = tfidf_vect.transform(text)

In [125]:
additional_features = dev_test_df[['text_length', 'roberta_sent_neg', 'roberta_sent_mixed',
                            'roberta_sent_pos', 'names', 'organizations', 'dates',
                            'count_tokens', 'cleaned_text_length', 'punctuation_count']].to_numpy()

In [126]:
embeddings_np = np.array(reduced_embeddings)
tfidf_features_np = tfidf_features.toarray()
additional_features_np = np.array(additional_features)

In [132]:
X = np.hstack([tfidf_features_np,embeddings_np, additional_features_np])
y = dev_test_df['label']

In [128]:
y_pred = model_rfc_us.predict(X)

In [133]:
print(classification_report(y, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.90      0.83       210
           1       0.72      0.48      0.58       108

    accuracy                           0.76       318
   macro avg       0.75      0.69      0.71       318
weighted avg       0.76      0.76      0.75       318



# test the dev_test data with the hyperparameter tuned model

In [ ]:
# tuned model without resampling



In [ ]:
def evaluate_the_model(model, test_features ,test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors/test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:.02f}%.'.format(accuracy))
    return accuracy